In [1]:
import pandas as pd
import plotly.express as px
from data_utils import read_rpt_file, replace_null_with_none, get_available_free_service_km, get_available_free_warranty_year, get_correct_dtype
from exploratory_utils import get_missing_percentage, compute_r_squared
from visualization import Plotting

# Configuration
from config import ModelVariables

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 200

In [2]:
responding_var = ModelVariables().responding_variable
exploration_vars = ModelVariables().numeric_variables + ModelVariables().categorical_variables + ModelVariables().date_variables

In [3]:
df = read_rpt_file('./data/DatiumTrain.rpt')

In [4]:
df.describe()

# Some columns are duplicated / equivalent to another (eg. Make vs MakeCode or Model vs FamilyCode)
# There are some columns having string 'NULL' instead of None. it need to be replaced.

,﻿Make,Model,MakeCode,FamilyCode,YearGroup,MonthGroup,SequenceNum,Description,CurrentRelease,ImportFlag,LimitedEdition,Series,SeriesModelYear,BadgeDescription,BadgeSecondaryDescription,BodyStyleDescription,BodyConfigDescription,WheelBaseConfig,Roofline,ExtraIdentification,DriveDescription,DriveCode,GearTypeDescription,GearLocationDescription,GearNum,DoorNum,EngineSize,EngineDescription,Cylinders,FuelTypeDescription,InductionDescription,OptionCategory,CamDescription,EngineTypeDescription,FuelCapacity,FuelDeliveryDescription,MethodOfDeliveryDescription,GrossCombinationMAss,GrossVehicleMass,VIN,WheelBase,Height,Length,Width,KerbWeight,TareMass,PayLoad,Power,PowerRPMFrom,PowerRPMTo,Torque,TorqueRPMFrom,TorqueRPMTo,RonRating,SeatCapacity,ModelCode,BuildCountryOriginDescription,ValvesCylinder,EngineCycleDescription,EngineConfigurationDescription,EngineLocation,EngineNum,Acceleration,FrontTyreSize,RearTyreSize,FrontRimDesc,RearRimDesc,TowingBrakes,TowingNoBrakes,WarrantyCustAssist,FreeScheduledService,WarrantyYears,WarrantyKM,FirstServiceKM,FirstServiceMonths,RegServiceMonths,AltEngEngineType,AltEngBatteryType,AltEngCurrentType,AltEngAmpHours,AltEngVolts,AltEngChargingMethod,AltEngPower,AltEngPowerFrom,AltEngPowerTo,AltEngTorque,AltEngTorqueFrom,AltEngTorqueTo,AltEngDrive,NormalChargeMins,QuickChargeMins,NormalChargeVoltage,QuickChargeVoltage,KMRangeElectricEng,ElectricEngineLocation,TopSpeedElectricEng,GreenhouseRating,AirpollutionRating,OverallGreenStarRating,CO2Combined,CO2Urban,CO2ExtraUrban,FuelUrban,FuelExtraurban,FuelCombined,EmissionStandard,MaxEthanolBlend,AncapRating,VFactsClass,VFactsSegment,VFactsPrice,IsPPlateApproved,AverageKM,GoodKM,AvgWholesale,AvgRetail,GoodWholesale,GoodRetail,TradeMin,TradeMax,PrivateMax,NewPrice,Colour,Branch,SaleCategory,Sold_Date,Compliance_Date,Age_Comp_Months,KM,Sold_Amount
count,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50704,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703,50703
unique,52,582,52,581,42,13,102,8568,1,1,2,1036,47,649,187,16,4,5,4,134,7,7,8,7,10,4,420,63,8,6,7,6,11,3,86,7,4,661,583,3979,339,452,739,287,1302,1160,853,232,15,65,331,45,84,4,13,3387,31,5,2,11,4,1522,133,296,314,70,83,161,72,60,8,7,14,18,9,7,3,3,3,3,9,4,12,3,6,13,1,5,4,1,1,1,1,2,3,2,17,14,11,272,293,166,183,95,134,6,4,6,5,14,18,2,90,78,752,840,800,910,422,440,520,4237,906,18,6,4747,405,278,43679,2716
top,Toyota,Commodore,TOYO,COMMODO,2010,0,1,ACV40R Altise Sedan 4dr Auto 5sp 2.4i,F,L,F,VE,NULL,NULL,NULL,Sedan,NULL,NULL,NULL,NULL,Rear Wheel Drive,RWD,Automatic,Floor,4,4,3984,4.0,4,Petrol - Unleaded ULP,Aspirated,PASS,DOHC with VVT,Piston,70,Multi-Point Injection,Electronic Sequential,NULL,NULL,6FPAAAJGSW3M#####,2775,1470,4815,1842,NULL,NULL,NULL,147.00,NULL,6000,380,NULL,4000,91,5,NULL,AUSTRALIA,4,4 Stroke,In-line,Front,VA-######,NULL,205/65 R15,205/65 R15,15x6.0,15x6.0,2100,750,NULL,NULL,3,100000,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,Passenger,Large,< $70K,T,180,140,NULL,NULL,NULL,3000,NULL,NULL,4100,29990,White,Belmore (NSW),Auction,2015-07-01 00:00:00.000,06/2010,38,0,0.00
freq,15720,6827,15720,6827,4200,37580,8218,707,50704,50704,49893,1713,35164,6605,48271,20594,42021,48905,50184,47447,21863,21863,22783,47630,2451

In [5]:
df.dtypes.head(20)

# We need to provide the dtypes explicitly to the variables

Make                        object
Model                        object
MakeCode                     object
FamilyCode                   object
YearGroup                    object
MonthGroup                   object
SequenceNum                  object
Description                  object
CurrentRelease               object
ImportFlag                   object
LimitedEdition               object
Series                       object
SeriesModelYear              object
BadgeDescription             object
BadgeSecondaryDescription    object
BodyStyleDescription         object
BodyConfigDescription        object
WheelBaseConfig              object
Roofline                     object
ExtraIdentification          object
dtype: object

In [6]:
# Missing data overview for each columns
df = replace_null_with_none(df)
missing_data_count = get_missing_percentage(df)
print(missing_data_count)

# Those columns with high missing data percentage are impossible to be impute with valid values and does not bring much insights. Thus, we will exclude them

                                Count  Missing Data Percentage
﻿Make                               0                 0.000000
Model                               0                 0.000000
MakeCode                            0                 0.000000
FamilyCode                          0                 0.000000
YearGroup                           0                 0.000000
MonthGroup                          0                 0.000000
SequenceNum                         0                 0.000000
Description                         0                 0.000000
CurrentRelease                      0                 0.000000
ImportFlag                          0                 0.000000
LimitedEdition                      0                 0.000000
Series                            850                 1.676396
SeriesModelYear                 35164                69.351530
BadgeDescription                 6605                13.026586
BadgeSecondaryDescription       48271                95

## Data Preprosessing

In [7]:
# Mileage is one key factor in determining the price, thus excluded if missing. Only two rows, so impact is minimal
df = df.loc[~df['KM'].isnull()]

#Exclude rows without sold amount too as it is essential
df = df.loc[~df['Sold_Amount'].isnull()]

In [8]:
# Lets reduce the df size
df_filt = df[exploration_vars + [responding_var]]

In [9]:
df_filt = get_correct_dtype(df=df_filt, numeric_vars=ModelVariables().numeric_variables + [responding_var], categorical_vars=ModelVariables().categorical_variables, date_vars=ModelVariables().date_variables)
df_filt.dtypes

c:\Datium\data_utils.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')
c:\Datium\data_utils.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('category')
c:\Datium\data_utils.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

Power                                   float64
NewPrice                                  int64
GearNum                                   int64
DoorNum                                   int64
Cylinders                                 int64
SeatCapacity                            float64
WarrantyYears                           float64
WarrantyKM                              float64
Age_Comp_Months                         float64
KM                                        int64
MakeCode                               category
FamilyCode                             category
EngineDescription                      category
FuelTypeDescription                    category
RonRating                              category
BuildCountryOriginDescription          category
SaleCategory                           category
Sold_Date                        datetime64[ns]
Sold_Amount                             float64
dtype: object

In [10]:
# Illustrate box plot for Sold_Amount

# Create a boxplot for Sold_Amount
fig = px.box(df_filt, y='Sold_Amount', title='Boxplot of Sold Amount')
fig.show()

In [11]:
# Assuming those missing data dont have warranty, thus replace None with 0
df_filt = get_available_free_warranty_year(df_filt)
df_filt = get_available_free_service_km(df_filt)

# Let's take the year of sold only
df_filt['Year_Sold'] = df_filt['Sold_Date'].dt.year

c:\Datium\data_utils.py:137: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Datium\data_utils.py:138: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Datium\data_utils.py:123: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Datium\data_utils.py:124: SettingWithCop

In [12]:
# Drop unwanted columns
df_processed = df_filt.drop(columns=ModelVariables().excluded_variables)  

# Feel the dataset

In [13]:
# Let's have a look at R2 coefficient
from exploratory_utils import compute_r_squared
r_squared = compute_r_squared(data = df_processed[["Sold_Amount", "Power", "NewPrice", "GearNum", "DoorNum", "Cylinders", "SeatCapacity", "AvailableWarrantyYears", "AvailableWarrantyKM", "Age_Comp_Months", "KM"]], target_variable=responding_var)
r_squared

,Variable,R-squared
0,Power,0.055135
1,NewPrice,0.419871
2,GearNum,0.030646
3,DoorNum,0.005289
4,Cylinders,0.022976
5,SeatCapacity,0.030864
6,AvailableWarrantyYears,0.000662
7,AvailableWarrantyKM,0.085723
8,Age_Comp_Months,0.198694
9,KM,0.131591


## Visualization

In [14]:
# Initializa Plot Class
PlotClass = Plotting

In [15]:
PlotClass.create_correlation_grid(data=df_processed, target_var="Sold_Amount")

# From the graph, we can observe that
# There are abnormal outliers for AvailableWarrantyKM - we need to remove the outliers
# There is 1 outlier for KM - 8Million+ KM, we need to remove that
# There is 1 outlier for Age_Comp_Months, we need to remove that

fig.write_html("docs/subset.html") 
# refer to subset.html in docs folder